# Feature Engineering

---

1. Import packages
2. Load data
3. Feature engineering

---

## 1. Import packages

In [2]:
import pandas as pd

---
## 2. Load data

In [3]:
df = pd.read_csv('./clean_data_after_eda.csv')
df["date_activ"] = pd.to_datetime(df["date_activ"], format='%Y-%m-%d')
df["date_end"] = pd.to_datetime(df["date_end"], format='%Y-%m-%d')
df["date_modif_prod"] = pd.to_datetime(df["date_modif_prod"], format='%Y-%m-%d')
df["date_renewal"] = pd.to_datetime(df["date_renewal"], format='%Y-%m-%d')

In [4]:
df.head(3)

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_off_peak_var,var_6m_price_peak_var,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,0.000131,4.100838e-05,0.000908,2.086294,99.530517,44.235794,2.086425,9.953056e+01,44.236702,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.000003,1.217891e-03,0.000000,0.009482,0.000000,0.000000,0.009485,1.217891e-03,0.000000,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.000004,9.450150e-08,0.000000,0.000000,0.000000,0.000000,0.000004,9.450150e-08,0.000000,0


---

## 3. Feature engineering

### Difference between off-peak prices in December and preceding January

Below is the code created by your colleague to calculate the feature described above. Use this code to re-create this feature and then think about ways to build on this feature to create features with a higher predictive power.

In [6]:
price_df = pd.read_csv('price_data (1).csv')
price_df["price_date"] = pd.to_datetime(price_df["price_date"], format='%Y-%m-%d')
price_df.head()

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0
3,038af19179925da21a25619c5a24b745,2015-04-01,0.149626,0.0,0.0,44.266931,0.0,0.0
4,038af19179925da21a25619c5a24b745,2015-05-01,0.149626,0.0,0.0,44.266931,0.0,0.0


In [7]:
# Group off-peak prices by companies and month
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_off_peak_var': 'mean', 'price_off_peak_fix': 'mean'}).reset_index()

# Get january and december prices
jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
dec_prices = monthly_price_by_id.groupby('id').last().reset_index()

# Calculate the difference
diff = pd.merge(dec_prices.rename(columns={'price_off_peak_var': 'dec_1', 'price_off_peak_fix': 'dec_2'}), jan_prices.drop(columns='price_date'), on='id')
diff['offpeak_diff_dec_january_energy'] = diff['dec_1'] - diff['price_off_peak_var']
diff['offpeak_diff_dec_january_power'] = diff['dec_2'] - diff['price_off_peak_fix']
diff = diff[['id', 'offpeak_diff_dec_january_energy','offpeak_diff_dec_january_power']]
diff.head()

,id,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,0002203ffbb812588b632b9e628cc38d,-0.006192,0.162916
1,0004351ebdd665e6ee664792efc4fd13,-0.004104,0.177779
2,0010bcc39e42b3c2131ed2ce55246e3c,0.050443,1.500000
3,0010ee3855fdea87602a5b7aba8e42de,-0.010018,0.162916
4,00114d74e963e47177db89bc70108537,-0.003994,-0.000001


Now it is time to get creative and to conduct some of your own feature engineering! Have fun with it, explore different ideas and try to create as many as you can!

#### Feature engineering Client Data

In [8]:
df.head()

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_off_peak_var,var_6m_price_peak_var,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,0.000131,4.100838e-05,9.084737e-04,2.086294,99.530517,44.235794,2.086425,9.953056e+01,4.423670e+01,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.000003,1.217891e-03,0.000000e+00,0.009482,0.000000,0.000000,0.009485,1.217891e-03,0.000000e+00,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.000004,9.450150e-08,0.000000e+00,0.000000,0.000000,0.000000,0.000004,9.450150e-08,0.000000e+00,0
3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,...,0.000003,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000003,0.000000e+00,0.000000e+00,0
4,149d57cf92fc41cf94415803a877cb4b,MISSING,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,...,0.000011,2.896760e-06,4.860000e-10,0.000000,0.000000,0.000000,0.000011,2.896760e-06,4.860000e-10,0


In [10]:
df.columns

Index(['id', 'channel_sales', 'cons_12m', 'cons_gas_12m', 'cons_last_month',
       'date_activ', 'date_end', 'date_modif_prod', 'date_renewal',
       'forecast_cons_12m', 'forecast_cons_year', 'forecast_discount_energy',
       'forecast_meter_rent_12m', 'forecast_price_energy_off_peak',
       'forecast_price_energy_peak', 'forecast_price_pow_off_peak', 'has_gas',
       'imp_cons', 'margin_gross_pow_ele', 'margin_net_pow_ele', 'nb_prod_act',
       'net_margin', 'num_years_antig', 'origin_up', 'pow_max',
       'var_year_price_off_peak_var', 'var_year_price_peak_var',
       'var_year_price_mid_peak_var', 'var_year_price_off_peak_fix',
       'var_year_price_peak_fix', 'var_year_price_mid_peak_fix',
       'var_year_price_off_peak', 'var_year_price_peak',
       'var_year_price_mid_peak', 'var_6m_price_off_peak_var',
       'var_6m_price_peak_var', 'var_6m_price_mid_peak_var',
       'var_6m_price_off_peak_fix', 'var_6m_price_peak_fix',
       'var_6m_price_mid_peak_fix', 'var_6m_p

In [11]:
# Importing necessary libraries
import numpy as np
from sklearn.preprocessing import StandardScaler
from datetime import datetime

In [12]:
# Convert date columns to datetime
date_cols = ['date_activ', 'date_end', 'date_modif_prod', 'date_renewal']
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')

In [13]:
df.head(3)

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_off_peak_var,var_6m_price_peak_var,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,0.000131,4.100838e-05,0.000908,2.086294,99.530517,44.235794,2.086425,9.953056e+01,44.236702,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.000003,1.217891e-03,0.000000,0.009482,0.000000,0.000000,0.009485,1.217891e-03,0.000000,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.000004,9.450150e-08,0.000000,0.000000,0.000000,0.000000,0.000004,9.450150e-08,0.000000,0


In [14]:
#  checking for missing values
df.duplicated().sum()

0

In [15]:
# Encode binary feature
df['has_gas'] = df['has_gas'].map({'t': 1, 'f': 0})

In [16]:
# Date-based features
today = pd.Timestamp(datetime.today())
df['days_active'] = (df['date_end'] - df['date_activ']).dt.days
df['days_since_modif'] = (df['date_renewal'] - df['date_modif_prod']).dt.days
df['days_until_renewal'] = (df['date_renewal'] - today).dt.days

In [17]:
# Extract date parts
for col in date_cols:
    df[f'{col}_year'] = df[col].dt.year
    df[f'{col}_month'] = df[col].dt.month
    df[f'{col}_dayofweek'] = df[col].dt.dayofweek

In [18]:
# Aggregate statistics on var_* columns
year_price_cols = [col for col in df.columns if 'var_year_price' in col]
six_month_cols = [col for col in df.columns if 'var_6m_price' in col]

df['std_yearly_price_var'] = df[year_price_cols].apply(pd.to_numeric, errors='coerce').std(axis=1)
df['mean_6m_price_fix'] = df[[col for col in six_month_cols if 'fix' in col]].apply(pd.to_numeric, errors='coerce').mean(axis=1)


df.head(2)

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,date_end_month,date_end_dayofweek,date_modif_prod_year,date_modif_prod_month,date_modif_prod_dayofweek,date_renewal_year,date_renewal_month,date_renewal_dayofweek,std_yearly_price_var,mean_6m_price_fix
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,6,2,2015,11,6,2015,6,1,21.022251,48.617535
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,8,1,2009,8,4,2015,8,0,0.002787,0.003161


In [19]:
# Interaction features
df['cons_pow_interaction'] = pd.to_numeric(df['cons_12m'], errors='coerce') * pd.to_numeric(df['pow_max'], errors='coerce')
df['margin_prod_interaction'] = pd.to_numeric(df['margin_net_pow_ele'], errors='coerce') * pd.to_numeric(df['nb_prod_act'], errors='coerce')

In [20]:
# Scale numerical features
exclude_cols = ['churn'] if 'churn' in df.columns else []
numerical_features = df.select_dtypes(include=[np.number]).drop(columns=exclude_cols, errors='ignore').columns
scaler = StandardScaler()
df[numerical_features] = scaler.fit_transform(df[numerical_features])

In [21]:
df.head(3)

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,date_modif_prod_year,date_modif_prod_month,date_modif_prod_dayofweek,date_renewal_year,date_renewal_month,date_renewal_dayofweek,std_yearly_price_var,mean_6m_price_fix,cons_pow_interaction,margin_prod_interaction
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,-0.277655,0.164779,-0.249996,2013-06-15,2016-06-15,2015-11-01,2015-06-23,-0.782669,...,1.024566,1.198433,1.842022,-0.152543,-0.178324,-0.818358,2.843715,4.385844,-0.202892,0.587952
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,-0.269529,-0.172380,-0.249996,2009-08-21,2016-08-30,2009-08-21,2015-08-31,-0.703109,...,-1.403078,0.295854,0.750226,-0.152543,0.391362,-1.294912,-0.152851,-0.092362,-0.199197,-0.466666
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,-0.276707,-0.172380,-0.249996,2010-04-16,2016-04-16,2010-04-16,2015-04-17,-0.762581,...,-0.998471,-0.907584,0.750226,-0.152543,-0.748011,0.611305,-0.152767,-0.092653,-0.202459,-0.093117


In [22]:
df['channel_sales'].nunique()

8

In [23]:
# One-hot encode categorical columns with int dtype
categorical_cols = ['channel_sales', 'origin_up']
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True, dtype=int)


In [24]:
df.shape

(14606, 73)

### Feature enginerering for price data

In [25]:
price_df.head(3)

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0


In [26]:
price_df.columns

Index(['id', 'price_date', 'price_off_peak_var', 'price_peak_var',
       'price_mid_peak_var', 'price_off_peak_fix', 'price_peak_fix',
       'price_mid_peak_fix'],
      dtype='object')

In [27]:
# Converting data type to datetime
price_df['price_date'] = pd.to_datetime(price_df['price_date'])

# Extract year, month, and day of week
price_df['year'] = price_df['price_date'].dt.year
price_df['month'] = price_df['price_date'].dt.month
price_df['day_of_week'] = price_df['price_date'].dt.dayofweek
price_df['is_weekend'] = price_df['day_of_week'].isin([5, 6]).astype(int)


In [28]:
price_df.dtypes

id                            object
price_date            datetime64[ns]
price_off_peak_var           float64
price_peak_var               float64
price_mid_peak_var           float64
price_off_peak_fix           float64
price_peak_fix               float64
price_mid_peak_fix           float64
year                           int32
month                          int32
day_of_week                    int32
is_weekend                     int64
dtype: object

In [29]:
# Creating average and total prices
# Average variable price
price_df['avg_var_price'] = price_df[['price_off_peak_var', 'price_peak_var', 'price_mid_peak_var']].mean(axis=1)

# Average fixed price
price_df['avg_fix_price'] = price_df[['price_off_peak_fix', 'price_peak_fix', 'price_mid_peak_fix']].mean(axis=1)

# Total variable and fixed prices
price_df['total_var_price'] = price_df[['price_off_peak_var', 'price_peak_var', 'price_mid_peak_var']].sum(axis=1)
price_df['total_fix_price'] = price_df[['price_off_peak_fix', 'price_peak_fix', 'price_mid_peak_fix']].sum(axis=1)


In [31]:
price_df.head(3)

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix,year,month,day_of_week,is_weekend,avg_var_price,avg_fix_price,total_var_price,total_fix_price
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0,2015,1,3,0,0.050456,14.755644,0.151367,44.266931
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0,2015,2,6,1,0.050456,14.755644,0.151367,44.266931
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0,2015,3,6,1,0.050456,14.755644,0.151367,44.266931


In [32]:
## Calculate peak to off peak ratios
# Avoid division by zero
price_df['var_peak_to_offpeak_ratio'] = price_df['price_peak_var'] / (price_df['price_off_peak_var'] + 1e-5)
price_df['fix_peak_to_offpeak_ratio'] = price_df['price_peak_fix'] / (price_df['price_off_peak_fix'] + 1e-5)


In [33]:
## Create price rage features(peak - off-peak)
price_df['var_range_peak_off'] = price_df['price_peak_var'] - price_df['price_off_peak_var']
price_df['fix_range_peak_off'] = price_df['price_peak_fix'] - price_df['price_off_peak_fix']


In [34]:
## Create volatility (range) features
# Variable price range (max - min)
price_df['var_price_volatility'] = price_df[['price_off_peak_var', 'price_peak_var', 'price_mid_peak_var']].max(axis=1) - \
                             price_df[['price_off_peak_var', 'price_peak_var', 'price_mid_peak_var']].min(axis=1)

# Fixed price range (max - min)
price_df['fix_price_volatility'] = price_df[['price_off_peak_fix', 'price_peak_fix', 'price_mid_peak_fix']].max(axis=1) - \
                             price_df[['price_off_peak_fix', 'price_peak_fix', 'price_mid_peak_fix']].min(axis=1)


In [35]:
price_df.head(3)

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix,year,month,...,avg_var_price,avg_fix_price,total_var_price,total_fix_price,var_peak_to_offpeak_ratio,fix_peak_to_offpeak_ratio,var_range_peak_off,fix_range_peak_off,var_price_volatility,fix_price_volatility
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0,2015,1,...,0.050456,14.755644,0.151367,44.266931,0.0,0.0,-0.151367,-44.266931,0.151367,44.266931
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0,2015,2,...,0.050456,14.755644,0.151367,44.266931,0.0,0.0,-0.151367,-44.266931,0.151367,44.266931
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0,2015,3,...,0.050456,14.755644,0.151367,44.266931,0.0,0.0,-0.151367,-44.266931,0.151367,44.266931


In [36]:
price_df.columns

Index(['id', 'price_date', 'price_off_peak_var', 'price_peak_var',
       'price_mid_peak_var', 'price_off_peak_fix', 'price_peak_fix',
       'price_mid_peak_fix', 'year', 'month', 'day_of_week', 'is_weekend',
       'avg_var_price', 'avg_fix_price', 'total_var_price', 'total_fix_price',
       'var_peak_to_offpeak_ratio', 'fix_peak_to_offpeak_ratio',
       'var_range_peak_off', 'fix_range_peak_off', 'var_price_volatility',
       'fix_price_volatility'],
      dtype='object')

In [37]:
df.columns

Index(['id', 'cons_12m', 'cons_gas_12m', 'cons_last_month', 'date_activ',
       'date_end', 'date_modif_prod', 'date_renewal', 'forecast_cons_12m',
       'forecast_cons_year', 'forecast_discount_energy',
       'forecast_meter_rent_12m', 'forecast_price_energy_off_peak',
       'forecast_price_energy_peak', 'forecast_price_pow_off_peak', 'has_gas',
       'imp_cons', 'margin_gross_pow_ele', 'margin_net_pow_ele', 'nb_prod_act',
       'net_margin', 'num_years_antig', 'pow_max',
       'var_year_price_off_peak_var', 'var_year_price_peak_var',
       'var_year_price_mid_peak_var', 'var_year_price_off_peak_fix',
       'var_year_price_peak_fix', 'var_year_price_mid_peak_fix',
       'var_year_price_off_peak', 'var_year_price_peak',
       'var_year_price_mid_peak', 'var_6m_price_off_peak_var',
       'var_6m_price_peak_var', 'var_6m_price_mid_peak_var',
       'var_6m_price_off_peak_fix', 'var_6m_price_peak_fix',
       'var_6m_price_mid_peak_fix', 'var_6m_price_off_peak',
       'var_6m

In [38]:
# Merging both dataframes together
# Merge on 'id' using a left join to keep all customer records
merged_df = df.merge(price_df, on='id', how='left')

In [40]:
merged_df.head(3)

,id,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,forecast_cons_year,...,avg_var_price,avg_fix_price,total_var_price,total_fix_price,var_peak_to_offpeak_ratio,fix_peak_to_offpeak_ratio,var_range_peak_off,fix_range_peak_off,var_price_volatility,fix_price_volatility
0,24011ae4ebbe3035111d65fa7c15bc57,-0.277655,0.164779,-0.249996,2013-06-15,2016-06-15,2015-11-01,2015-06-23,-0.782669,-0.431005,...,0.100302,27.043980,0.300907,81.131939,0.820686,0.6,-0.022581,-16.226389,0.05444,24.339581
1,24011ae4ebbe3035111d65fa7c15bc57,-0.277655,0.164779,-0.249996,2013-06-15,2016-06-15,2015-11-01,2015-06-23,-0.782669,-0.431005,...,0.100302,27.043980,0.300907,81.131939,0.820686,0.6,-0.022581,-16.226389,0.05444,24.339581
2,24011ae4ebbe3035111d65fa7c15bc57,-0.277655,0.164779,-0.249996,2013-06-15,2016-06-15,2015-11-01,2015-06-23,-0.782669,-0.431005,...,0.100302,27.043978,0.300907,81.131934,0.820686,0.6,-0.022581,-16.226395,0.05444,24.339590


In [41]:
# selecting relevant columns
# Define relevant features from df
df_features = [
    'cons_12m', 'cons_gas_12m', 'cons_last_month', 'forecast_cons_12m',
    'forecast_cons_year', 'forecast_discount_energy', 'forecast_meter_rent_12m',
    'forecast_price_energy_off_peak', 'forecast_price_energy_peak',
    'forecast_price_pow_off_peak', 'has_gas', 'imp_cons',
    'margin_gross_pow_ele', 'margin_net_pow_ele', 'nb_prod_act', 'net_margin',
    'num_years_antig', 'pow_max', 'std_yearly_price_var', 'mean_6m_price_fix',
    'cons_pow_interaction', 'margin_prod_interaction',
    'days_active', 'days_since_modif', 'days_until_renewal'
]

# Add categorical one-hot columns
df_features += [col for col in df.columns if col.startswith('channel_sales_') or col.startswith('origin_up_')]

# Add engineered price features from price_df
price_features = [
    'year', 'month', 'day_of_week', 'is_weekend',
    'avg_var_price', 'avg_fix_price', 'total_var_price', 'total_fix_price',
    'var_peak_to_offpeak_ratio', 'fix_peak_to_offpeak_ratio',
    'var_range_peak_off', 'fix_range_peak_off',
    'var_price_volatility', 'fix_price_volatility'
]

# Combine all features
selected_columns = df_features + price_features + ['churn']


In [42]:
# Final dataset for prediction
final_df = merged_df[selected_columns]

In [43]:
# Check for any missing values introduced during the merge
print(final_df.isnull().sum().sort_values(ascending=False))

cons_12m                                          0
cons_gas_12m                                      0
channel_sales_foosdfpfkusacimwkcsosbicdxkicaua    0
channel_sales_lmkebamcaaclubfxadlmueccxoimlema    0
channel_sales_sddiedcslfslkckwlfkdpoeeailfpeds    0
channel_sales_usilxuppasemubllopkaafesmlibmsdf    0
origin_up_ewxeelcelemmiwuafmddpobolfuxioce        0
origin_up_kamkkxfxxuwbdslkwifmmcsiusiuosws        0
origin_up_ldkssxwpmemidmecebumciepifcamkci        0
origin_up_lxidpiddsbxsbosboudacockeimpuepw        0
origin_up_usapbepcfoloekilkwsdiboslwaxobdp        0
year                                              0
month                                             0
day_of_week                                       0
is_weekend                                        0
avg_var_price                                     0
avg_fix_price                                     0
total_var_price                                   0
total_fix_price                                   0
var_peak_to_

In [44]:
final_df.shape

(175149, 52)

In [45]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175149 entries, 0 to 175148
Data columns (total 52 columns):
 #   Column                                          Non-Null Count   Dtype  
---  ------                                          --------------   -----  
 0   cons_12m                                        175149 non-null  float64
 1   cons_gas_12m                                    175149 non-null  float64
 2   cons_last_month                                 175149 non-null  float64
 3   forecast_cons_12m                               175149 non-null  float64
 4   forecast_cons_year                              175149 non-null  float64
 5   forecast_discount_energy                        175149 non-null  float64
 6   forecast_meter_rent_12m                         175149 non-null  float64
 7   forecast_price_energy_off_peak                  175149 non-null  float64
 8   forecast_price_energy_peak                      175149 non-null  float64
 9   forecast_price_pow_off_pea